In [5]:
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from datetime import datetime, date
import time
import matplotlib.pyplot as plt

### Read In Crime data table

In [6]:
df_train = pd.read_csv('./data/interim/crime_data/fullTrainTestCrimeData_train_police_rand_2019_06_27.csv') 
df_valid = pd.read_csv('./data/interim/crime_data/fullTrainTestCrimeData_valid_police_rand_2019_06_27.csv') 
df_tests = pd.read_csv('./data/interim/crime_data/fullTrainTestCrimeData_test_police_rand_2019_06_27.csv')

In [7]:
df_valid.head(5)

,Unnamed: 0,DATE,DAY_OF_MONTH,DAY_OF_WEEK,Hour_Grp,INCIDENT_NUMBER,Lat,Long,MONTH,Value,YEAR,closest_mbta_d,closest_police_d
0,0,2018-04-01,1,Sunday,1,[],42.2348,-71.1325,4,0,2018,3.563674,1.599552
1,1,2018-04-01,1,Sunday,2,[],42.2348,-71.1325,4,0,2018,3.563674,1.599552
2,2,2018-04-01,1,Sunday,3,[],42.2348,-71.1325,4,0,2018,3.563674,1.599552
3,3,2018-04-02,2,Monday,1,[],42.2348,-71.1325,4,0,2018,3.563674,1.599552
4,4,2018-04-02,2,Monday,2,[],42.2348,-71.1325,4,0,2018,3.563674,1.599552


In [8]:
df_train = df_train.drop(columns=["Unnamed: 0"])
df_valid = df_valid.drop(columns=["Unnamed: 0"])
df_tests = df_tests.drop(columns=["Unnamed: 0"])

### Create the column DAY_OF_WEEK

In [9]:
map_day = {'Monday':1, 'Tuesday':2,'Wednesday':3, 'Thursday':4, 'Friday':5, 
           'Saturday':6, 'Sunday':7}
df_train["DAY_OF_WEEK_NUM"] = df_train["DAY_OF_WEEK"].apply(lambda x: map_day[x])
df_valid["DAY_OF_WEEK_NUM"] = df_valid["DAY_OF_WEEK"].apply(lambda x: map_day[x])
df_tests["DAY_OF_WEEK_NUM"] = df_tests["DAY_OF_WEEK"].apply(lambda x: map_day[x])

### Read In the Weather info

In [10]:
df_weather_all_by_day = pd.read_csv('./data/interim/weather/weather_all_by_day_hourgroup.csv')
df_weather_all_by_day = df_weather_all_by_day.drop(columns=["Unnamed: 0"])
print(df_weather_all_by_day.shape)

(4865, 10)


#### More cleaning of weather table

In [11]:
if df_weather_all_by_day[df_weather_all_by_day["DATE"] == '2018-08-26'].shape[0] <3:
    new_series = df_weather_all_by_day[df_weather_all_by_day["DATE"] == '2018-08-26'].iloc[0]
    new_series["Hour_Grp"] = 1
    df_weather_all_by_day = df_weather_all_by_day.append(new_series , ignore_index=True)
df_weather_all_by_day[df_weather_all_by_day["DATE"] == '2018-08-26']

,DATE,Hour_Grp,Temperature(F),Dew Point(F),Humidity(%),Wind Speed(mph),Wind Gust(mph),Pressure(in),Precip.(in),Precip Accum(in)
3999,2018-08-26,2,76.000000,66.0,71.0,12.000000,0.0,30.0,0.0,0.0
4000,2018-08-26,3,74.666667,67.0,77.0,10.666667,0.0,30.0,0.0,0.0
4865,2018-08-26,1,76.000000,66.0,71.0,12.000000,0.0,30.0,0.0,0.0


In [12]:
print (df_weather_all_by_day.shape)

(4866, 10)


### Mearge Weather with Crime

In [13]:
merged_df_train = pd.merge(df_train, df_weather_all_by_day,  how='left', left_on=['DATE','Hour_Grp'], right_on = ['DATE','Hour_Grp'])
merged_df_valid = pd.merge(df_valid, df_weather_all_by_day,  how='left', left_on=['DATE','Hour_Grp'], right_on = ['DATE','Hour_Grp'])
merged_df_tests = pd.merge(df_tests, df_weather_all_by_day,  how='left', left_on=['DATE','Hour_Grp'], right_on = ['DATE','Hour_Grp'])
print (merged_df_train.shape)
print (merged_df_valid.shape)
print (merged_df_tests.shape)

(787191, 21)
(164352, 21)
(164808, 21)


In [15]:
merged_df_train.to_csv('./data/processed/crime_data/fullTrainTestCrimeData_train_weather_by_hourgroup_police_rand_2019_06_27.csv')
merged_df_valid.to_csv('./data/processed/crime_data/fullTrainTestCrimeData_valid_weather_by_hourgroup_police_rand_2019_06_27.csv')
merged_df_tests.to_csv('./data/processed/crime_data/fullTrainTestCrimeData_tests_weather_by_hourgroup_police_rand_2019_06_27.csv')